In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[18]:


from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
import cv2  
import matplotlib.pyplot as plt
import socket
from _thread import *


# define function to load train, test, and validation datasets
def load_dataset(path):
    '''
    Load dataset
    '''
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 24)
    # print(data['filenames'])
    return dog_files, dog_targets


# load train, test, and validation datasets
train_files, train_targets = load_dataset('data/train')
valid_files, valid_targets = load_dataset('data/valid')
test_files, test_targets = load_dataset('data/test')

# load list of dog names
dog_names = [item[11:-1] for item in sorted(glob("data/train/*/"))]

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.' % len(test_files))

# In[2]:


from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras

restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, output=output)
for layer in restnet.layers:
    layer.trainable = False
# restnet.summary()


# In[3]:


from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers

model = Sequential()

model.add(restnet)

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(24, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['accuracy'])
# model.summary()


# In[4]:


from keras.preprocessing import image
from tqdm import tqdm


def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)


def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)


# In[5]:


from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

model.load_weights('saved_models/weights.hdf5')

# In[9]:


from keras.applications.resnet50 import preprocess_input, decode_predictions

ResNet50_model = ResNet50(weights='imagenet')

def ResNet50_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(ResNet50_model.predict(img))


def dog_detector(img_path):
    prediction = ResNet50_predict_labels(img_path)
    return ((prediction <= 268) & (prediction >= 151))


def dog_breed(img_path):
    predicted_vector = model.predict(path_to_tensor(img_path))  # shape error occurs here
    # return dog breed that is predicted by the model
    # print(predicted_vector)
    fir = dog_names[np.argmax(predicted_vector)]
    fir_per = np.max(predicted_vector) * 100
    predicted_vector = np.delete(predicted_vector, np.argmax(predicted_vector))

    sec = dog_names[np.argmax(predicted_vector)]
    sec_per = np.max(predicted_vector) * 100
    return fir, fir_per, sec, sec_per


def getLength(client_socket):
    header = client_socket.recv(2)
    return header[0] + (header[1] << 8)

def threaded(client_socket):
    global index
    while True:
        try:
            data = client_socket.recv(4);
            if not data:
                break
            length = int.from_bytes(data, "little");
            stringData = b''
            while length:
                newbuf = client_socket.recv(length)
                if not newbuf: return None
                stringData += newbuf
                length -= len(newbuf)

            data = np.frombuffer(stringData, dtype='uint8')
            img = cv2.imdecode(data, 1)
            #cv2.imshow('kk', img)
            fileName = str(index) + '.jpg'
            cv2.imwrite(fileName, img)
            if dog_detector(fileName):
                img = cv2.imread(fileName)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                plt.imshow(img);
                print(fileName)
                breed_1, per1, breed_2, per2 = dog_breed(fileName)
                persum = per1 + per2
                msg = 'this dog is,' + str(breed_1, per1 / persum * 100) + '% or ' + str(breed_2) + str(per2 / persum * 100) + "%"
            else:
                img = cv2.imread(fileName)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                plt.imshow(img);
                print('This is not a dog.')
                breed_1, per1, breed_2, per2 = dog_breed(fileName)
                persum = per1 + per2
                msg = 'If it is dog,' + str(breed_1, per1 / persum * 100) + '% or ' + str(breed_2) + str(per2 / persum * 100) + "%"
            #data = msg.encode();
            #length = len(data);
            #client_socket.sendall(length.to_bytes(4, byteorder='little'));
            #client_socket.sendall(data);
            print(msg)
            index += 1
        except:
            break

    client_socket.close()


index = 0

HOST = '' # 건들지 마세요
PORT = 1319 # 포트

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_socket.bind((HOST, PORT))
server_socket.listen()

print("열림")
while True:
    client_socket, _ = server_socket.accept()
    print("접속")
    start_new_thread(threaded, (client_socket,))

server_socket.close()





Using TensorFlow backend.


There are 24 total dog categories.
There are 7535 total dog images.

There are 5923 training dog images.
There are 806 validation dog images.
There are 806 test dog images.


C:\Users\user\anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:57: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=Tensor("fl...)`


열림
접속
접속
접속
접속
접속
접속
